# 🤖 LangGraph 多 Agent 系统实战

本 Notebook 展示如何使用 LangGraph 构建复杂的多 Agent 协作系统，并通过 LangSmith 进行详细的监控和调试。

## 📋 学习目标

- LangGraph 基础概念（State、Node、Edge）
- 构建状态机工作流
- 多 Agent 协作模式
- 条件路由和决策
- **使用 LangSmith 监控每个 Agent 的执行细节**
- 实战案例：智能内容创作团队

## 💡 LangSmith 监控特性

在本 Notebook 中，你将能在 LangSmith 上看到：
- ✅ 每个 Agent 的执行时间
- ✅ 每个 Agent 的输入状态和输出状态
- ✅ Agent 之间的调用关系
- ✅ 每次 LLM 调用的完整 Prompt 和响应
- ✅ Token 使用和成本统计


## 📦 安装依赖


In [1]:
%pip install -q langgraph langchain langchain-openai langsmith


Note: you may need to restart the kernel to use updated packages.


## 🔧 配置环境变量

**系统环境变量**（已在 `~/.zshrc` 中配置）：
- `LANGCHAIN_API_KEY` - LangSmith API Key
- `OPENAI_API_KEY` - DeepSeek API Key

**项目环境变量**（自动设置）：
- `LANGCHAIN_TRACING_V2=true` - 启用 LangSmith 追踪
- `LANGCHAIN_PROJECT=langgraph-multi-agent` - 项目名称
- `OPENAI_API_BASE=https://api.deepseek.com` - DeepSeek API 地址


In [2]:
import os
from typing import TypedDict, Annotated, Sequence
from langchain_openai import ChatOpenAI

print("🔧 配置环境")
print("="*60)

# 自动设置项目环境变量
os.environ.setdefault("LANGCHAIN_TRACING_V2", "true")
os.environ.setdefault("LANGCHAIN_PROJECT", "langgraph-multi-agent")
os.environ.setdefault("OPENAI_API_BASE", "https://api.deepseek.com")

# 检查系统环境变量
api_key = os.environ.get("OPENAI_API_KEY", "")
langsmith_key = os.environ.get("LANGCHAIN_API_KEY", "")

print("\n📋 配置状态：")
print("-"*60)

if api_key:
    print(f"✅ OPENAI_API_KEY: {api_key[:8]}...{api_key[-4:]} (系统)")
else:
    print("❌ OPENAI_API_KEY: 未设置")

print(f"✅ OPENAI_API_BASE: {os.environ['OPENAI_API_BASE']} (项目)")

if langsmith_key:
    print(f"✅ LANGCHAIN_API_KEY: {langsmith_key[:12]}...{langsmith_key[-4:]} (系统)")
else:
    print("❌ LANGCHAIN_API_KEY: 未设置")

print(f"✅ LANGCHAIN_TRACING_V2: {os.environ['LANGCHAIN_TRACING_V2']} (项目)")
print(f"✅ LANGCHAIN_PROJECT: {os.environ['LANGCHAIN_PROJECT']} (项目)")

print("-"*60)

if not api_key or not langsmith_key:
    print("\n⚠️  请确保在 ~/.zshrc 中配置了必需的环境变量")
    print("   然后重启 Jupyter Kernel")
else:
    print("\n✅ 所有配置就绪！")
    print(f"✅ 访问 https://smith.langchain.com/ 查看执行详情")

print("="*60)

# 初始化 LLM
llm = ChatOpenAI(
    model="deepseek-chat",
    openai_api_base=os.environ["OPENAI_API_BASE"],
    temperature=0.7
)

print("\n✅ LLM 初始化完成！")


🔧 配置环境

📋 配置状态：
------------------------------------------------------------
✅ OPENAI_API_KEY: sk-9d725...6c29 (系统)
✅ OPENAI_API_BASE: https://api.deepseek.com (项目)
✅ LANGCHAIN_API_KEY: lsv2_pt_9d1d...4df1 (系统)
✅ LANGCHAIN_TRACING_V2: true (项目)
✅ LANGCHAIN_PROJECT: langgraph-multi-agent (项目)
------------------------------------------------------------

✅ 所有配置就绪！
✅ 访问 https://smith.langchain.com/ 查看执行详情

✅ LLM 初始化完成！


## 一、多 Agent 内容创作系统

### 1.1 定义状态


In [3]:
from typing import TypedDict, List
import operator

class AgentState(TypedDict):
    """多 Agent 系统的共享状态"""
    topic: str                    # 主题
    research_notes: str           # 研究笔记
    draft_content: str            # 草稿
    final_content: str            # 最终内容
    messages: Annotated[List, operator.add]  # 执行日志
    next_agent: str               # 下一个 Agent
    iteration: int                # 当前迭代次数
    max_iterations: int           # 最大迭代次数

print("✅ AgentState 定义完成")


✅ AgentState 定义完成


### 1.2 创建 Agent（带完整追踪）

每个 Agent 使用 `@traceable` 装饰器，在 LangSmith 中可以看到：
- 🔍 Agent 的输入状态
- 🔍 Agent 的输出状态  
- 🔍 Agent 内部的所有 LLM 调用
- 🔍 执行时间和 Token 使用


In [4]:
from langchain.prompts import ChatPromptTemplate
from langsmith import traceable

# ============================================================================
# 研究员 Agent
# ============================================================================

@traceable(
    name="🔬 研究员 Agent",
    run_type="chain",
    tags=["agent", "researcher", "content-creation"],
    metadata={"role": "researcher", "task": "research_and_analysis"}
)
def researcher_agent(state: AgentState) -> AgentState:
    """
    研究员 Agent：负责收集和分析信息
    
    在 LangSmith 中可以看到：
    - Input: topic
    - Output: research_notes
    - 执行时间
    - LLM 调用详情
    """
    
    print(f"\n{'='*60}")
    print(f"🔬 [研究员 Agent] 开始执行")
    print(f"{'='*60}")
    print(f"📥 输入 - 主题: {state['topic']}")
    
    prompt = ChatPromptTemplate.from_template(
        """你是一个专业的研究员。请针对以下主题进行研究，提供关键要点和有价值的信息。

主题：{topic}

请提供：
1. 核心概念定义
2. 重要特点（3-5个）
3. 实际应用场景
4. 注意事项

研究笔记："""
    )
    
    chain = prompt | llm
    result = chain.invoke(
        {"topic": state["topic"]},
        config={
            "metadata": {
                "agent": "researcher",
                "step": "research",
                "topic": state["topic"],
                "phase": "data_collection"
            },
            "tags": ["research_phase"],
            "run_name": f"📚 研究: {state['topic'][:30]}"
        }
    )
    
    print(f"📤 输出 - 生成了 {len(result.content)} 字的研究笔记")
    print(f"✅ [研究员 Agent] 执行完成")
    print(f"{'='*60}\n")
    
    return {
        **state,
        "research_notes": result.content,
        "messages": [f"[研究员] 完成研究: {state['topic'][:50]}..."],
        "next_agent": "writer"
    }


# ============================================================================
# 作家 Agent
# ============================================================================

@traceable(
    name="✍️ 作家 Agent",
    run_type="chain",
    tags=["agent", "writer", "content-creation"],
    metadata={"role": "writer", "task": "content_writing"}
)
def writer_agent(state: AgentState) -> AgentState:
    """
    作家 Agent：负责撰写文章
    
    在 LangSmith 中可以看到：
    - Input: research_notes
    - Output: draft_content
    - 执行时间
    - LLM 调用详情
    """
    
    print(f"\n{'='*60}")
    print(f"✍️ [作家 Agent] 开始执行")
    print(f"{'='*60}")
    print(f"📥 输入 - 研究笔记长度: {len(state['research_notes'])} 字")
    
    prompt = ChatPromptTemplate.from_template(
        """你是一个专业的技术作家。基于研究员提供的笔记，撰写一篇结构清晰、通俗易懂的文章。

主题：{topic}

研究笔记：
{research_notes}

要求：
- 使用清晰的标题和段落结构
- 语言通俗易懂，适合初学者
- 包含实例说明
- 字数约500字

文章内容："""
    )
    
    chain = prompt | llm
    result = chain.invoke(
        {
            "topic": state["topic"],
            "research_notes": state["research_notes"]
        },
        config={
            "metadata": {
                "agent": "writer",
                "step": "writing",
                "research_notes_length": len(state["research_notes"]),
                "phase": "content_creation"
            },
            "tags": ["writing_phase"],
            "run_name": f"📝 撰写: {state['topic'][:30]}"
        }
    )
    
    print(f"📤 输出 - 生成了 {len(result.content)} 字的文章草稿")
    print(f"✅ [作家 Agent] 执行完成")
    print(f"{'='*60}\n")
    
    return {
        **state,
        "draft_content": result.content,
        "messages": [f"[作家] 完成初稿撰写"],
        "next_agent": "editor"
    }


# ============================================================================
# 编辑 Agent
# ============================================================================

@traceable(
    name="📝 编辑 Agent",
    run_type="chain",
    tags=["agent", "editor", "content-creation"],
    metadata={"role": "editor", "task": "review_and_improve"}
)
def editor_agent(state: AgentState) -> AgentState:
    """
    编辑 Agent：负责审核和改进文章
    
    在 LangSmith 中可以看到：
    - Input: draft_content
    - Output: final_content
    - 当前迭代次数
    - 执行时间
    - LLM 调用详情
    """
    
    print(f"\n{'='*60}")
    print(f"📝 [编辑 Agent] 开始执行")
    print(f"{'='*60}")
    print(f"📥 输入 - 草稿长度: {len(state['draft_content'])} 字")
    print(f"🔄 当前迭代: {state['iteration'] + 1}/{state['max_iterations']}")
    
    prompt = ChatPromptTemplate.from_template(
        """你是一个专业的编辑。请审核并改进以下文章。

原文：
{draft_content}

请：
1. 检查语法和表达
2. 优化结构和逻辑
3. 确保内容准确性
4. 增强可读性

如果文章质量已经很好，保持原样并添加简短评价。
如果需要改进，请直接输出改进后的版本。

最终文章："""
    )
    
    chain = prompt | llm
    result = chain.invoke(
        {"draft_content": state["draft_content"]},
        config={
            "metadata": {
                "agent": "editor",
                "step": "editing",
                "iteration": state["iteration"] + 1,
                "max_iterations": state["max_iterations"],
                "draft_length": len(state["draft_content"]),
                "phase": "quality_control"
            },
            "tags": ["editing_phase"],
            "run_name": f"🔍 编辑: 第{state['iteration'] + 1}轮"
        }
    )
    
    new_iteration = state["iteration"] + 1
    
    print(f"📤 输出 - 最终文章长度: {len(result.content)} 字")
    print(f"✅ [编辑 Agent] 执行完成")
    print(f"{'='*60}\n")
    
    return {
        **state,
        "final_content": result.content,
        "messages": [f"[编辑] 完成审核和改进"],
        "next_agent": "END",
        "iteration": new_iteration
    }


print("✅ Agent 定义完成！")
print("\n💡 每个 Agent 都使用 @traceable 装饰器")
print("   在 LangSmith 中可以看到完整的执行过程")


✅ Agent 定义完成！

💡 每个 Agent 都使用 @traceable 装饰器
   在 LangSmith 中可以看到完整的执行过程


### 1.3 构建工作流


In [5]:
from langgraph.graph import StateGraph, END

# 路由函数：决定下一个节点
def route_agent(state: AgentState) -> str:
    next_agent = state.get("next_agent", "researcher")
    if next_agent == "END":
        return END
    return next_agent

# 创建图
workflow = StateGraph(AgentState)

# 添加节点
workflow.add_node("researcher", researcher_agent)
workflow.add_node("writer", writer_agent)
workflow.add_node("editor", editor_agent)

# 设置入口
workflow.set_entry_point("researcher")

# 添加条件边
workflow.add_conditional_edges(
    "researcher",
    route_agent,
    {"writer": "writer", END: END}
)

workflow.add_conditional_edges(
    "writer",
    route_agent,
    {"editor": "editor", END: END}
)

workflow.add_conditional_edges(
    "editor",
    route_agent,
    {"writer": "writer", END: END}
)

# 编译
app = workflow.compile()

print("✅ 工作流构建完成！")
print("\n📊 工作流结构：")
print("   研究员 → 作家 → 编辑 → 结束")


✅ 工作流构建完成！

📊 工作流结构：
   研究员 → 作家 → 编辑 → 结束


### 1.4 运行多 Agent 系统

**在 LangSmith 中查看：**
1. 访问 https://smith.langchain.com/
2. 选择项目：`langgraph-multi-agent`
3. 找到最新的 Run
4. 展开树形结构，你将看到：
   - 🔬 研究员 Agent（包含输入输出）
   - ✍️ 作家 Agent（包含输入输出）
   - 📝 编辑 Agent（包含输入输出）
   - 每个 Agent 内部的 LLM 调用详情


In [6]:
# 初始化状态
initial_state = {
    "topic": "深度学习中的 Transformer 架构",
    "research_notes": "",
    "draft_content": "",
    "final_content": "",
    "messages": [],
    "next_agent": "researcher",
    "iteration": 0,
    "max_iterations": 2
}

print("\n" + "="*80)
print("🚀 启动多 Agent 内容创作系统")
print("="*80)
print(f"\n📝 主题: {initial_state['topic']}")
print(f"\n💡 执行过程将实时上传到 LangSmith")
print(f"   访问 https://smith.langchain.com/ 查看详细的执行追踪")
print(f"\n🔍 在 LangSmith 中你将看到：")
print(f"   - 每个 Agent 的执行时间")
print(f"   - 每个 Agent 的输入和输出")
print(f"   - 所有 LLM 调用的 Prompt 和响应")
print(f"   - Token 使用和成本统计")
print("\n" + "="*80 + "\n")

# 运行工作流
final_state = app.invoke(initial_state)

print("\n" + "="*80)
print("🎉 内容创作完成！")
print("="*80)

print("\n📋 执行流程:")
for msg in final_state["messages"]:
    print(f"  ✓ {msg}")

print(f"\n📊 统计信息:")
print(f"  - 研究笔记: {len(final_state['research_notes'])} 字")
print(f"  - 文章草稿: {len(final_state['draft_content'])} 字")
print(f"  - 最终文章: {len(final_state['final_content'])} 字")
print(f"  - 迭代次数: {final_state['iteration']}/{final_state['max_iterations']}")

print("\n" + "="*80)
print("📄 最终文章预览（前 500 字）:")
print("="*80)
print(final_state["final_content"][:500] + "...\n")

print("="*80)
print("🔍 查看完整执行详情：")
print("="*80)
print(f"\n1. 访问 LangSmith: https://smith.langchain.com/")
print(f"2. 选择项目: langgraph-multi-agent")
print(f"3. 找到最新的 Run（按时间排序）")
print(f"4. 点击展开，你会看到：")
print(f"\n   StateGraph Execution")
print(f"   ├── 🔬 研究员 Agent")
print(f"   │   ├── Inputs: {{topic: \"{initial_state['topic'][:30]}...\"}}")
print(f"   │   ├── 📚 研究: ... (LLM 调用)")
print(f"   │   └── Outputs: {{research_notes: \"...\"}}")
print(f"   ├── ✍️ 作家 Agent")
print(f"   │   ├── Inputs: {{research_notes: \"...\"}}")
print(f"   │   ├── 📝 撰写: ... (LLM 调用)")
print(f"   │   └── Outputs: {{draft_content: \"...\"}}")
print(f"   └── 📝 编辑 Agent")
print(f"       ├── Inputs: {{draft_content: \"...\"}}")
print(f"       ├── 🔍 编辑: ... (LLM 调用)")
print(f"       └── Outputs: {{final_content: \"...\"}}")
print("\n" + "="*80)



🚀 启动多 Agent 内容创作系统

📝 主题: 深度学习中的 Transformer 架构

💡 执行过程将实时上传到 LangSmith
   访问 https://smith.langchain.com/ 查看详细的执行追踪

🔍 在 LangSmith 中你将看到：
   - 每个 Agent 的执行时间
   - 每个 Agent 的输入和输出
   - 所有 LLM 调用的 Prompt 和响应
   - Token 使用和成本统计



🔬 [研究员 Agent] 开始执行
📥 输入 - 主题: 深度学习中的 Transformer 架构
📤 输出 - 生成了 3105 字的研究笔记
✅ [研究员 Agent] 执行完成


✍️ [作家 Agent] 开始执行
📥 输入 - 研究笔记长度: 3105 字
📤 输出 - 生成了 1329 字的文章草稿
✅ [作家 Agent] 执行完成


📝 [编辑 Agent] 开始执行
📥 输入 - 草稿长度: 1329 字
🔄 当前迭代: 1/2
📤 输出 - 最终文章长度: 1643 字
✅ [编辑 Agent] 执行完成


🎉 内容创作完成！

📋 执行流程:
  ✓ [研究员] 完成研究: 深度学习中的 Transformer 架构...
  ✓ [作家] 完成初稿撰写
  ✓ [编辑] 完成审核和改进

📊 统计信息:
  - 研究笔记: 3105 字
  - 文章草稿: 1329 字
  - 最终文章: 1643 字
  - 迭代次数: 1/2

📄 最终文章预览（前 500 字）:
### **Transformer：让AI真正“读懂”上下文的革命性架构**

在人工智能领域，尤其是在语言理解与生成任务中，Transformer架构的提出堪称一场根本性变革。它不仅是ChatGPT、文心一言等智能助手的核心引擎，更彻底重塑了机器处理序列信息的方式。

---

#### **一、什么是Transformer？**

想象一下，你在阅读一句话时，不是逐词顺序理解，而是能瞬间把握所有词语之间的关联——这正是Transformer的核心能力。

该架构由Google团队于2017年提出，完全摒弃了传统的循环处理模式，转而采用名为“自注意力”（Self-Attention）的创新机制。简单来说，自注意力

## 二、总结

### ✅ 本 Notebook 的关键改进

**1. 完整的 Agent 可见性**
- 使用 `@traceable` 装饰器让每个 Agent 在 LangSmith 中可见
- 可以看到每个 Agent 的输入和输出
- 可以看到 Agent 之间的调用关系

**2. 详细的调试信息**
- 每个 LLM 调用都有 metadata 和 tags
- 自定义的 run_name 使追踪更直观
- 本地打印语句显示实时执行进度

**3. LangSmith 监控能力**
- 📊 完整的执行树形结构
- ⏱️ 每个 Agent 和 LLM 调用的耗时
- 💰 Token 使用和成本统计
- 🔍 完整的 Prompt 和响应内容
- 🏷️ 通过 tags 筛选和分类

### 🎯 掌握的技能

1. **LangGraph 多 Agent 系统**
   - State 状态管理
   - Agent 定义和协作
   - 条件路由和工作流编排

2. **LangSmith 深度集成**
   - `@traceable` 装饰器的使用
   - Metadata 和 Tags 的配置
   - 执行追踪的最佳实践

3. **调试和优化**
   - 完整的执行可见性
   - 性能瓶颈识别
   - 成本分析和优化

### 🚀 应用场景

- 📝 内容创作团队（本案例）
- 🎯 客户服务系统
- 🔍 代码审查流程
- 📊 数据分析管道
- 🤖 智能助手系统

### 💡 最佳实践

1. **使用 @traceable 装饰器**：让自定义函数在 LangSmith 中可见
2. **添加详细的 metadata**：便于后续分析和筛选
3. **使用有意义的名称**：emoji + 描述性名称
4. **添加本地日志**：本地和云端都能看到执行过程
5. **合理使用 tags**：便于分类和筛选

---

**🎉 恭喜！你现在可以在 LangSmith 中完整追踪多 Agent 系统的执行过程了！**

**📍 下一步：**
- 访问 LangSmith 查看你的执行记录
- 尝试修改 Agent 逻辑
- 添加更多 Agent
- 实现循环和条件路由
